In [1]:
from sagas.ofbiz.entities import OfEntity as e, oc, finder
hub=oc.component('entity_event_hub')
forms=oc.component('form_mgr')

# form_loc="component://content/widget/forum/BlogForms.xml;EditBlog;en_US"
# form_loc='component://party/widget/partymgr/LookupForms.xml;LookupPartyName;en_US'
form_loc='component://party/widget/partymgr/LookupForms.xml;LookupPartyName;zh'
form=forms.getMetaForm(form_loc)
print(form)

name: "LookupPartyName"
target: "LookupPartyName"
type: "single"
fields {
  name: "partyId"
  title: "\344\274\232\345\221\230\346\240\207\350\257\206"
  fieldType: TEXTQBE
  titleOriginal: "${uiLabelMap.PartyPartyId}"
}
fields {
  name: "partyTypeId"
  title: "\344\274\232\345\221\230\347\261\273\345\236\213\346\240\207\350\257\206"
  fieldType: DROP_DOWN
  titleOriginal: "${uiLabelMap.PartyTypeId}"
  dropDownField {
    allowEmpty: true
    current: "first-in-list"
    size: "1"
    textSize: "0"
  }
}
fields {
  name: "firstName"
  title: "\345\220\215\345\255\227"
  fieldType: TEXTQBE
  titleOriginal: "${uiLabelMap.PartyFirstName}"
}
fields {
  name: "lastName"
  title: "\345\247\223\346\260\217"
  fieldType: TEXTQBE
  titleOriginal: "${uiLabelMap.PartyLastName}"
}
fields {
  name: "groupName"
  title: "\347\273\204\345\220\215\347\247\260"
  fieldType: TEXTQBE
  titleOriginal: "${uiLabelMap.PartyGroupName}"
}
fields {
  name: "noConditionFind"
  title: "No Condition Find"
  fieldT

In [10]:
from forms_pb2 import MetaForm, MetaMappingPackage, MetaFieldMapping, MetaFieldMappings, SUBMIT, RESET

def extract_key(original):
    return original.replace('${uiLabelMap.', '').replace('}', '').strip()
    
py_form=MetaForm()
form_data=form.toByteString().toByteArray()
py_form.ParseFromString(form_data)
package={}
for fld in py_form.fields:
    if fld.titleOriginal is not None and len(fld.titleOriginal)>0 and fld.fieldType not in (SUBMIT, RESET):                
        key=extract_key(fld.titleOriginal)
        print(fld.name, '♯', fld.title, fld.titleOriginal, '♯', key)
        mapping=MetaFieldMapping(key=key, fieldName=fld.name, 
                                 fieldTitle=fld.title, 
                                 fieldTitleOriginal=fld.titleOriginal,
                                 formUri=form_loc
                                )
        if key in package:
            package[key].fields.append(mapping)
            # print('+', package[key])
        else:
            # print('add', key)
            package[key]=MetaFieldMappings(fields=[mapping])

meta_package=MetaMappingPackage(mappings=package)
print(meta_package)

partyId ♯ 会员标识 ${uiLabelMap.PartyPartyId} ♯ PartyPartyId
partyTypeId ♯ 会员类型标识 ${uiLabelMap.PartyTypeId} ♯ PartyTypeId
firstName ♯ 名字 ${uiLabelMap.PartyFirstName} ♯ PartyFirstName
lastName ♯ 姓氏 ${uiLabelMap.PartyLastName} ♯ PartyLastName
groupName ♯ 组名称 ${uiLabelMap.PartyGroupName} ♯ PartyGroupName
mappings {
  key: "PartyFirstName"
  value {
    fields {
      key: "PartyFirstName"
      fieldName: "firstName"
      fieldTitle: "\345\220\215\345\255\227"
      fieldTitleOriginal: "${uiLabelMap.PartyFirstName}"
      formUri: "component://party/widget/partymgr/LookupForms.xml;LookupPartyName;zh"
    }
  }
}
mappings {
  key: "PartyGroupName"
  value {
    fields {
      key: "PartyGroupName"
      fieldName: "groupName"
      fieldTitle: "\347\273\204\345\220\215\347\247\260"
      fieldTitleOriginal: "${uiLabelMap.PartyGroupName}"
      formUri: "component://party/widget/partymgr/LookupForms.xml;LookupPartyName;zh"
    }
  }
}
mappings {
  key: "PartyLastName"
  value {
    fields {
  

In [6]:
def component_loc(loc):
    prefix="component://"
    pkg_prefixes = ['ofbiz-framework/applications',
                    'ofbiz-framework/framework',
                    'ofbiz-framework/plugins']
    for pkg_prefix in pkg_prefixes:
        idx = loc.find(pkg_prefix)
        if idx != -1:
            return prefix+loc[idx + len(pkg_prefix) + 1:]
    raise ValueError('Cannot normalize the location ' + loc)

component_loc('/Users/xiaofeiwu/jcloud/vagrant/fedora/fedora-28/ofbiz/ofbiz-framework/applications/workeffort/widget/TimesheetForms.xml')

'component://workeffort/widget/TimesheetForms.xml'

In [1]:
from sagas.ofbiz.util import norm_loc
norm_loc('/Users/xiaofeiwu/jcloud/vagrant/fedora/fedora-28/ofbiz/ofbiz-framework/applications/workeffort/widget/TimesheetForms.xml')

'workeffort_timesheetforms'

In [4]:
from sagas.ofbiz.entity_prefabs import all_components
all_components()

base
entity
security
datafile
minilang
common
service
catalina
entityext
webapp
widget
testtools
webtools
bluelight
common-theme
flatgrey
rainbowstone
tomahawk
datamodel
party
securityext
content
workeffort
product
manufacturing
accounting
humanres
order
marketing
commonext
ecommerce
example
ofbizDemo
projectmgr
sagas


In [2]:
class FormResource(object):
    def __init__(self, comp, name, location):
        self.comp=comp
        self.name=name
        self.location=location
        
        self.forms=[]

class FormDescriptor(object):
    def __init__(self, tag, name, type, target, extends):
        self.tag=tag
        self.name=name
        self.type=type
        self.target=target
        self.extends=extends

In [6]:
from sagas.ofbiz.entities import OfEntity as e, oc, finder
import os
import io_utils
oc.import_package('org.apache.ofbiz.base.component.ComponentConfig')
allComponents = oc.j.ComponentConfig.getAllComponents()
form_list=[]
for c in allComponents:
    # print(c.getRootLocation())
    widget_dir=c.getRootLocation()+"widget"
    if os.path.isdir(widget_dir):
        files=io_utils.list_files(widget_dir)
        print(c.getGlobalName(), len(files))
        counts={'forms':0, 'screens':0, 'menus':0, 'trees':0, 'others':0}
        for f in files:
            base=os.path.basename(f)
            name=os.path.splitext(base)[0]
            if 'Form' in name:
                counts['forms']=counts['forms']+1
                form_list.append(FormResource(c.getGlobalName(), name, f))
            elif 'Screen' in name:
                counts['screens']=counts['screens']+1
            elif 'Menu' in name:
                counts['menus']=counts['menus']+1
            elif 'Tree' in name:
                counts['trees']=counts['trees']+1
            elif name=='Theme':
                pass
            else:
                counts['others']=counts['others']+1
                print('** get unexpected file type', name)
        print('\t', counts)

print(len(form_list))

common 12
	 {'forms': 4, 'screens': 6, 'menus': 2, 'trees': 0, 'others': 0}
webtools 24
	 {'forms': 10, 'screens': 13, 'menus': 1, 'trees': 0, 'others': 0}
bluelight 1
	 {'forms': 0, 'screens': 0, 'menus': 0, 'trees': 0, 'others': 0}
common-theme 4
	 {'forms': 0, 'screens': 3, 'menus': 0, 'trees': 0, 'others': 0}
flatgrey 1
	 {'forms': 0, 'screens': 0, 'menus': 0, 'trees': 0, 'others': 0}
rainbowstone 1
	 {'forms': 0, 'screens': 0, 'menus': 0, 'trees': 0, 'others': 0}
tomahawk 1
	 {'forms': 0, 'screens': 0, 'menus': 0, 'trees': 0, 'others': 0}
party 20
	 {'forms': 8, 'screens': 11, 'menus': 1, 'trees': 0, 'others': 0}
securityext 1
	 {'forms': 0, 'screens': 1, 'menus': 0, 'trees': 0, 'others': 0}
content 40
** get unexpected file type CMSTemplates
	 {'forms': 13, 'screens': 15, 'menus': 9, 'trees': 2, 'others': 1}
workeffort 22
	 {'forms': 12, 'screens': 7, 'menus': 2, 'trees': 1, 'others': 0}
product 48
	 {'forms': 20, 'screens': 24, 'menus': 3, 'trees': 1, 'others': 0}
manufacturing 

In [46]:
import xml.etree.ElementTree as ET

total=0
form_index={}
for form_res in form_list:
    tree = ET.parse(form_res.location)
    root = tree.getroot()
    for child in root:
        # tag, name, type, target        
        fd=FormDescriptor(child.tag, child.get('name'), 
                          child.get('type'), 
                          child.get('target'),
                          child.get('extends')
                         )
        form_res.forms.append(fd)
        if fd.name not in form_index:
            form_index[fd.name]=[form_res]
        else:
            form_index[fd.name].append(form_res)
            print('duplicate form name %s, extends -> %s'%(fd.name, fd.extends))
        total=total+1
print(total)        

duplicate form name ListPortalPages, extends -> None
duplicate form name LookupUserLogin, extends -> None
duplicate form name ListLookedUpUserLogins, extends -> None
duplicate form name ListLookupGeo, extends -> ListGeos
duplicate form name AddUserLogin, extends -> None
duplicate form name UpdatePassword, extends -> None
duplicate form name UpdateUserLoginSecurity, extends -> None
duplicate form name ListWebSites, extends -> None
duplicate form name FindContent, extends -> None
duplicate form name EditContent, extends -> None
duplicate form name AddContentPurposeOperation, extends -> None
duplicate form name UpdateContentPurposeOperation, extends -> None
duplicate form name ListWorkEffortCommEvents, extends -> None
duplicate form name LookupCommEvent, extends -> None
duplicate form name ListLookupCommEvent, extends -> None
duplicate form name AddWorkEffortContent, extends -> None
duplicate form name ListWorkEffortContents, extends -> None
duplicate form name ListRequests, extends -> No

In [39]:
import json
form=form_list[0]
print(form.name, form.location)
# print(form.forms)
printstr=json.dumps(form, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)
print(printstr)

PortalPageForms /Users/xiaofeiwu/jcloud/vagrant/fedora/fedora-28/ofbiz/ofbiz-framework/framework/common/widget/PortalPageForms.xml
{
    "comp": "common",
    "forms": [
        {
            "extends": null,
            "name": "ListPortalPages",
            "tag": "{http://ofbiz.apache.org/Widget-Form}form",
            "target": null,
            "type": "list"
        },
        {
            "extends": null,
            "name": "NewPortalPage",
            "tag": "{http://ofbiz.apache.org/Widget-Form}form",
            "target": "createPortalPage",
            "type": "single"
        },
        {
            "extends": null,
            "name": "PortletCategoryAndPortlet",
            "tag": "{http://ofbiz.apache.org/Widget-Form}form",
            "target": null,
            "type": "list"
        },
        {
            "extends": null,
            "name": "PortletList",
            "tag": "{http://ofbiz.apache.org/Widget-Form}form",
            "target": null,
            "typ

In [41]:
locs=form_index['ListTaskInfo']
for loc in locs:
    print(loc.name, loc.location)

ProjectForms /Users/xiaofeiwu/jcloud/vagrant/fedora/fedora-28/ofbiz/ofbiz-framework/plugins/projectmgr/widget/forms/ProjectForms.xml
ProjectForms /Users/xiaofeiwu/jcloud/vagrant/fedora/fedora-28/ofbiz/ofbiz-framework/plugins/projectmgr/widget/forms/ProjectForms.xml


In [47]:
# AddTimesheetEntry
locs=form_index['AddTimesheetEntry']
for loc in locs:
    print(loc.name, loc.location)

TimesheetForms /Users/xiaofeiwu/jcloud/vagrant/fedora/fedora-28/ofbiz/ofbiz-framework/applications/workeffort/widget/TimesheetForms.xml
TimeSheetForms /Users/xiaofeiwu/jcloud/vagrant/fedora/fedora-28/ofbiz/ofbiz-framework/plugins/projectmgr/widget/forms/TimeSheetForms.xml
